[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2021/blob/main/99--exercises/42--copmare-performance.ipynb)

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz -O spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [1]:
!pip install -q pyspark findspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Test it")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

21/12/11 07:49:39 WARN Utils: Your hostname, Jakubs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.8.8 instead (on interface en0)
21/12/11 07:49:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/11 07:49:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/11 07:49:40 WARN Utils: Service 'SparkUI' could not bind on port 4050. Attempting port 4051.


In [17]:
sc = spark.sparkContext

In [3]:
from pyspark.sql import functions as F

In [4]:
df_by_age = spark.read.parquet("employees-by-age.parquet")

In [5]:
df_by_age.show()

+-------------+----+
|wynagrodzenie|wiek|
+-------------+----+
|        12600|  65|
|        13300|  65|
|        13400|  65|
|        12800|  65|
|        12600|  65|
|        13000|  65|
|        13400|  65|
|        12800|  65|
|        12700|  65|
|        13300|  65|
|        13200|  65|
|        12500|  65|
|        13400|  65|
|        13200|  65|
|        12900|  65|
|        12900|  65|
|        12600|  65|
|        13100|  65|
|        13400|  65|
|        13200|  65|
+-------------+----+
only showing top 20 rows



In [9]:
rm -rf employee-by-age.json employee-by-sallary.json

In [10]:
df_by_age.repartition(1)\
    .write\
    .partitionBy('wiek')\
    .format('json')\
    .save('employee-by-age.json')

In [11]:
df_by_age.repartition(30)\
    .write\
    .partitionBy('wynagrodzenie')\
    .format('json')\
    .save('employee-by-sallary.json')

In [12]:
df = spark.read.json("employee-by-age.json/")

In [13]:
%%time
df.groupby('wiek').count().show()

+----+-----+
|wiek|count|
+----+-----+
|  65|   72|
|  53|   48|
|  28|   55|
|  26|   61|
|  44|   48|
|  47|   51|
|  52|   54|
|  40|   55|
|  57|   48|
|  54|   55|
|  48|   50|
|  64|   48|
|  43|   46|
|  37|   45|
|  61|   47|
|  35|   47|
|  55|   44|
|  39|   58|
|  51|   42|
|  50|   54|
+----+-----+
only showing top 20 rows

CPU times: user 1.92 ms, sys: 1.21 ms, total: 3.13 ms
Wall time: 398 ms


In [14]:
df = spark.read.json("employee-by-sallary.json/")

In [15]:
%%time
df.groupby('wiek').count().show()

+----+-----+
|wiek|count|
+----+-----+
|  29|   64|
|  54|   55|
|  34|   37|
|  57|   48|
|  32|   41|
|  43|   46|
|  31|   42|
|  39|   58|
|  51|   42|
|  56|   60|
|  52|   54|
|  41|   43|
|  33|   53|
|  48|   50|
|  44|   48|
|  61|   47|
|  62|   52|
|  49|   41|
|  35|   47|
|  36|   50|
+----+-----+
only showing top 20 rows

CPU times: user 9.47 ms, sys: 4.68 ms, total: 14.2 ms
Wall time: 1.81 s


In [18]:
sc.stop()